
# 🌼 대규모 LLM을 활용한 지식 챗봇 개발 - 3차시(24.11.25)

---

In [1]:
import numpy as np
import random
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [2]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [3]:
sent = '오늘 밥 뭐먹지'

In [4]:
input_ids = tokenizer.encode(sent)

In [5]:
input_ids

[10070, 14821, 46651, 7513, 8263]

In [6]:
input_ids = tf.convert_to_tensor([input_ids])
# 정수 리스트를 텐서로 변환

In [7]:
input_ids.shape
# 2차원 텐서로 변환되었다
# (1, 5): (입력 데이터의 배치 크기, 문장의 토큰 수)

TensorShape([1, 5])

In [8]:
output = model.generate(input_ids, max_length=128, repetition_penalty=2.0, use_cache=True)
# max_length: 출력의 최대 길이를 128 토큰으로
# repetition penalty: 동일한 단어나 문장이 계속 생성되는 것을 억제 (1.0~2.0 사이를 많이 쓴다/1.0은 패널티가 없고 2.0은 강한 억제)
# use_cache: 이전에 생성됐던 문장을 생성해내는 캐시 값을 사용 (반복되는 것과는 다름)

In [9]:
output_ids = output.numpy().tolist()[0]  # numpy > 리스트로 변환

In [10]:
print(output_ids)
# 내가 입력한 문장으로 시작한다

[10070, 14821, 46651, 7513, 8263, 10604, 7965, 7172, 7991, 389, 9050, 468, 10723, 9585, 6947, 7401, 9022, 6853, 9784, 8017, 8006, 13675, 25830, 9105, 7512, 7177, 406, 9873, 7098, 8022, 8046, 9182, 7722, 10078, 8146, 8196, 9265, 7235, 12857, 9018, 8456, 11732, 15495, 13358, 14782, 8139, 9252, 7220, 45887, 42430, 739, 7570, 17003, 6969, 31279, 10666, 9330, 11403, 9242, 26389, 9036, 9515, 33146, 7662, 9668, 11355, 8155, 7208, 9329, 12002, 19561, 7788, 25689, 10917, 12341, 24923, 8135, 24107, 48541, 9122, 27511, 8084, 32539, 16203, 9651, 31204, 25856, 9755, 7532, 9239, 10587, 9572, 15709, 10162, 15740, 13088, 12036, 9278, 15882, 10586, 9031, 9144, 7982, 9887, 7162, 9108, 9306, 21855, 18948, 9025, 9080, 10185, 37375, 8162, 8030, 15247, 10910, 10401, 32109, 10165, 16285, 49542, 9277, 9193, 6824, 22386, 23854, 9625]


In [11]:
tokenizer.decode(output_ids)

'오늘 밥 뭐먹지?\n아니야. 아~ 그래.\n그럼 그거 먹었어요?\n네. 어머님? 네네에예 예뻐요.\n이제 저도 이제 이케 좀 먹고 싶어요.\n음 근데 제가 인제 뭘 먹을까 고민하다가 음 그래서 요거를 한 번 먹어봤는데 맛있더라고요\n그래서 그냥 이렇게 해서 먹으면은 되게 맛있는 거 같아요 라고 생각을 했거든요. 라면 같은 경우에는 사실 우리가 보통 먹는 음식 중에 가장 맛이 좋은 것 같애 가지고는 또 다른 맛을 낼 수 있는 그런 음식이잖여 맞아 그런데 왜 굳이 이런 음식을 먹어야 하는 건가 싶은 생각이 들'

In [12]:
output = model(input_ids)

In [14]:
top5 = tf.math.top_k(output.logits[0, -1], k=5)
top5

TopKV2(values=<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([13.177099, 12.499327, 11.421543, 10.66643 , 10.644171],
      dtype=float32)>, indices=<tf.Tensor: shape=(5,), dtype=int32, numpy=array([10604,   406, 46651, 37202, 10723], dtype=int32)>)

In [15]:
tokenizer.convert_ids_to_tokens(top5.indices.numpy())  # 어떤 단어가 다음 단어일 확률 top5인지

['?\n', '?', '▁뭐', '▁그랬', '▁그래']

In [22]:
top5_values_list = top5.values.numpy().tolist()
top5_values_list

[14.184578895568848,
 14.142895698547363,
 13.89085865020752,
 11.839044570922852,
 11.451498031616211]

In [17]:
sent = '나는 집에'

input_ids = tokenizer.encode(sent)

while len(input_ids) < 50:
  output = model(np.array([input_ids]))
  top5 = tf.math.top_k(output.logits[0, -1], k=5)
  token_id = random.choice(top5.indices.numpy())  # 가장 확률이 높은 값들 중 랜덤으로 다음 단어를 선택
  input_ids.append(token_id)
tokenizer.decode(input_ids)

'나는 집에 있는 것 같아요. 그리고 내가 이게 어떻게 생겼나요 하면 이거 이~ 이케 좀 뭐 이쁘다고 할 수도 있죠. 이거 이거 저~ 이게 뭐~ 저거 저~ 뭐야 저게 이'

In [21]:
sent = '지금까지'

input_ids = tokenizer.encode(sent)

while len(input_ids) < 50:
  # 텍스트 총 길이(토큰 개수)가 50개 미만인 동안 새로운 단어 추가
  output = model(np.array([input_ids]))
  top1 = tf.math.top_k(output.logits[0, -1], k=1)
  token_id = random.choice(top1.indices.numpy())
  input_ids.append(token_id)

tokenizer.decode(input_ids)

'지금까지 지난해 12월 말 기준 국내 총생산(GDP) 대비 국가채무 비율은 40.9%로 경제협력개발기구(OECD) 회원국 중 가장 높은 수준이다.\n이 같은 국가채무 비율은 OECD 평균인 40.9%보다 높은 수준이다.\n이 같은'